# Grouping and Aggregating Data

**Grouping and aggregating** are fundamental operations in data analysis that allow you to summarize and extract insights from your data. These techniques are particularly powerful when working with large datasets, enabling you to uncover patterns, trends, and statistics within different subsets of your data.


In Pandas, grouping refers to the process of splitting the data into groups based on some criteria. Aggregating involves computing a summary statistic (or statistics) about each group. Together, these operations allow you to answer questions like:

- What is the average sale price for each product category?
- How many sales were made in each region?
- What is the total revenue per quarter?


The primary method for grouping in Pandas is the `groupby()` function. It creates a `GroupBy` object, which can then be used with various aggregation functions to compute results.


Key concepts we'll cover in this lecture include:

- Using `groupby()` to create groups
- Applying aggregation functions to grouped data
- Working with multiple grouping columns
- Performing custom aggregations
- Handling time series and categorical data in groups
- Advanced grouping techniques and transformations


Let's start by importing the necessary libraries and creating a sample dataset of sales data to work with throughout this lecture:


In [9]:
import pandas as pd
import numpy as np

In [10]:
# Create a sample DataFrame of sales data
np.random.seed(0)
dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')
products = ['Laptop', 'Smartphone', 'Tablet', 'Headphones']
regions = ['North', 'South', 'East', 'West']

In [11]:
df = pd.DataFrame({
    'Date': np.random.choice(dates, 1000),
    'Product': np.random.choice(products, 1000),
    'Region': np.random.choice(regions, 1000),
    'Sales': np.random.randint(100, 1500, 1000),
    'Units': np.random.randint(1, 10, 1000)
})
df

,Date,Product,Region,Sales,Units
0,2023-06-22,Headphones,West,613,8
1,2023-02-17,Laptop,North,467,7
2,2023-04-28,Laptop,North,1315,7
3,2023-07-12,Laptop,East,1317,7
4,2023-11-20,Smartphone,North,1401,1
...,...,...,...,...,...
995,2023-07-03,Laptop,North,1282,1
996,2023-04-24,Laptop,East,333,7
997,2023-07-25,Headphones,North,1224,4
998,2023-11-08,Headphones,East,960,3


In [12]:
df['Revenue'] = df['Sales'] * df['Units']
df = df.sort_values('Date').reset_index(drop=True)
df

,Date,Product,Region,Sales,Units,Revenue
0,2023-01-01,Tablet,North,102,6,612
1,2023-01-01,Laptop,West,1051,9,9459
2,2023-01-01,Laptop,North,1235,4,4940
3,2023-01-01,Laptop,East,1238,1,1238
4,2023-01-03,Laptop,East,194,1,194
...,...,...,...,...,...,...
995,2023-12-30,Headphones,East,573,6,3438
996,2023-12-30,Smartphone,East,896,2,1792
997,2023-12-30,Smartphone,West,362,3,1086
998,2023-12-31,Laptop,South,283,1,283


In [13]:
df.head(10)

,Date,Product,Region,Sales,Units,Revenue
0,2023-01-01,Tablet,North,102,6,612
1,2023-01-01,Laptop,West,1051,9,9459
2,2023-01-01,Laptop,North,1235,4,4940
3,2023-01-01,Laptop,East,1238,1,1238
4,2023-01-03,Laptop,East,194,1,194
5,2023-01-03,Tablet,West,703,3,2109
6,2023-01-04,Laptop,North,677,6,4062
7,2023-01-04,Tablet,West,497,9,4473
8,2023-01-04,Laptop,East,523,9,4707
9,2023-01-04,Tablet,South,475,6,2850


This DataFrame contains:
- A 'Date' column representing the sale date
- A 'Product' column with different product categories
- A 'Region' column indicating where the sale occurred
- A 'Sales' column with the price per unit
- A 'Units' column showing the number of units sold
- A 'Revenue' column calculated as Sales * Units


We'll use this dataset to demonstrate various grouping and aggregation techniques throughout the lecture. This data represents a year's worth of sales information for an electronics retailer across different products and regions.


By working with this meaningful dataset, you'll see how grouping and aggregating can provide valuable insights such as:

- Total revenue by product category
- Average sales price in each region
- Monthly sales trends
- Best-selling products by units sold


By the end of this session, you'll have a solid understanding of how to use these powerful tools to analyze and summarize your data effectively, extracting business-relevant insights from raw sales data.

**Table of contents**<a id='toc0_'></a>    
- [Basic Grouping with `groupby()`](#toc1_)    
  - [Grouping by a Single Column](#toc1_1_)    
  - [Grouping by Multiple Columns](#toc1_2_)    
  - [Grouping with a Custom Function](#toc1_3_)    
- [Aggregation Methods](#toc2_)    
  - [Built-in Aggregation Functions](#toc2_1_)    
  - [Custom Aggregation Functions](#toc2_2_)    
  - [Multiple Aggregations](#toc2_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_'></a>[Basic Grouping with `groupby()`](#toc0_)

The `groupby()` function is the foundation of grouping operations in Pandas. It allows you to split your data into groups based on one or more columns, or even custom functions.


### <a id='toc1_1_'></a>[Grouping by a Single Column](#toc0_)


Let's start by grouping our sales data by product to see total revenue for each product category:


In [14]:
# Group by Product and sum the Revenue
product_revenue = df.groupby('Product')['Revenue'].sum()

product_revenue

Product
Headphones    1119087
Laptop        1055232
Smartphone    1135676
Tablet         805445
Name: Revenue, dtype: int64

This gives us a Series with the total revenue for each product. We can also use multiple aggregation functions:


In [15]:
# Multiple aggregations
product_summary = df.groupby('Product').agg({
    'Revenue': 'sum',
    'Units': 'sum',
    'Sales': 'mean'
})

product_summary

,Revenue,Units,Sales
Product,,,
Headphones,1119087,1373,789.369231
Laptop,1055232,1303,810.361868
Smartphone,1135676,1355,814.746094
Tablet,805445,1068,763.612335


### <a id='toc1_2_'></a>[Grouping by Multiple Columns](#toc0_)


We can group by multiple columns to get more detailed insights:


In [16]:
# Group by Product and Region
df.groupby(['Product', 'Region'])['Revenue'].sum()

Product     Region
Headphones  East      339165
            North     219987
            South     214237
            West      345698
Laptop      East      258699
            North     242457
            South     318475
            West      235601
Smartphone  East      300088
            North     275026
            South     336204
            West      224358
Tablet      East      175186
            North     188136
            South     227453
            West      214670
Name: Revenue, dtype: int64

This creates a hierarchical index. To make it easier to work with, we can use the `unstack()` method:


In [17]:
product_region_sales.unstack(level='Region')


Region,East,North,South,West
Product,,,,
Headphones,339165,219987,214237,345698
Laptop,258699,242457,318475,235601
Smartphone,300088,275026,336204,224358
Tablet,175186,188136,227453,214670


### <a id='toc1_3_'></a>[Grouping with a Custom Function](#toc0_)


Sometimes, we need to group by a criterion that's not directly available in our columns. We can use a custom function for this:


In [18]:
# Custom function to group by quarter
def get_quarter(date):
    return f'Q{date.quarter}'

In [19]:
# Group by quarter
quarterly_revenue = df.groupby(df['Date'].apply(get_quarter))['Revenue'].sum()

quarterly_revenue

Date
Q1     988223
Q2    1173587
Q3     938852
Q4    1014778
Name: Revenue, dtype: int64

We can also use lambda functions for simple operations:


In [20]:
# Group by month
monthly_revenue = df.groupby(df['Date'].dt.to_period('M'))['Revenue'].sum()

monthly_revenue

Date
2023-01    271227
2023-02    353486
2023-03    363510
2023-04    441359
2023-05    371862
2023-06    360366
2023-07    332699
2023-08    292041
2023-09    314112
2023-10    336301
2023-11    299121
2023-12    379356
Freq: M, Name: Revenue, dtype: int64

Here's an example combining custom grouping with multiple columns:


In [23]:
# Group by quarter and product
quarter_product_revenue = df.groupby([df['Date'].apply(get_quarter), 'Product'])['Revenue'].sum().unstack()

quarter_product_revenue

Product,Headphones,Laptop,Smartphone,Tablet
Date,,,,
Q1,277670,271099,264338,175116
Q2,289153,321356,352141,210937
Q3,254339,215515,293220,175778
Q4,297925,247262,225977,243614


These grouping techniques allow you to slice and dice your data in various ways, providing different perspectives on your dataset. Remember that after grouping, you typically need to apply some aggregation function (like `sum()`, `mean()`, `count()`, etc.) to get meaningful results.


The `groupby()` function is incredibly flexible, allowing you to group by:
- A single column
- Multiple columns
- A mix of columns and computed values
- Custom functions


This flexibility makes it a powerful tool for data analysis, enabling you to explore your data from multiple angles and uncover insights that might not be immediately apparent in the raw data.

## <a id='toc2_'></a>[Aggregation Methods](#toc0_)

After grouping data, we typically want to compute summary statistics for each group. Pandas provides a variety of methods for aggregating grouped data.


### <a id='toc2_1_'></a>[Built-in Aggregation Functions](#toc0_)


Pandas offers many built-in aggregation functions that can be applied directly to grouped data:


In [28]:
df

,Date,Product,Region,Sales,Units,Revenue
0,2023-01-01,Tablet,North,102,6,612
1,2023-01-01,Laptop,West,1051,9,9459
2,2023-01-01,Laptop,North,1235,4,4940
3,2023-01-01,Laptop,East,1238,1,1238
4,2023-01-03,Laptop,East,194,1,194
...,...,...,...,...,...,...
995,2023-12-30,Headphones,East,573,6,3438
996,2023-12-30,Smartphone,East,896,2,1792
997,2023-12-30,Smartphone,West,362,3,1086
998,2023-12-31,Laptop,South,283,1,283


In [24]:
# Group by Product
grouped = df.groupby('Product')

# Various built-in aggregations
built_in_aggs = grouped['Revenue'].agg([
    'count',    # Number of sales
    'sum',      # Total revenue
    'mean',     # Average revenue per sale
    'median',   # Median revenue
    'min',      # Minimum revenue
    'max',      # Maximum revenue
    'std'       # Standard deviation of revenue
])

built_in_aggs

,count,sum,mean,median,min,max,std
Product,,,,,,,
Headphones,260,1119087,4304.180769,3498.0,105,13473,3306.421445
Laptop,257,1055232,4105.961089,3420.0,176,13239,3080.226782
Smartphone,256,1135676,4436.234375,3641.5,102,13257,3357.415499
Tablet,227,805445,3548.215859,2790.0,102,13302,2904.353198


You can also use these functions on multiple columns at once:


In [25]:
grouped[['Revenue', 'Units']].agg(['sum', 'mean', 'max'])

Revenue                     Units              
                sum         mean    max   sum      mean max
Product                                                    
Headphones  1119087  4304.180769  13473  1373  5.280769   9
Laptop      1055232  4105.961089  13239  1303  5.070039   9
Smartphone  1135676  4436.234375  13257  1355  5.292969   9
Tablet       805445  3548.215859  13302  1068  4.704846   9

### <a id='toc2_2_'></a>[Custom Aggregation Functions](#toc0_)


For more complex calculations, you can define your own aggregation functions:


In [31]:
def revenue_per_unit(x):
    return x.sum() / df.loc[x.index, 'Units'].sum()

def top_sale(x):
    return x.max()

custom_aggs = grouped.agg({
    'Revenue': ['sum', revenue_per_unit],
    'Units': 'sum',
    'Sales': [top_sale, 'mean']
})

custom_aggs

Revenue                  Units    Sales            
                sum revenue_per_unit   sum top_sale        mean
Product                                                        
Headphones  1119087       815.067007  1373     1497  789.369231
Laptop      1055232       809.848043  1303     1497  810.361868
Smartphone  1135676       838.137269  1355     1496  814.746094
Tablet       805445       754.161985  1068     1496  763.612335

You can also use lambda functions for simple custom aggregations:


In [32]:
lambda_aggs = grouped['Revenue'].agg([
    ('Total', 'sum'),
    ('Average', 'mean'),
    ('Range', lambda x: x.max() - x.min())
])

lambda_aggs

,Total,Average,Range
Product,,,
Headphones,1119087,4304.180769,13368
Laptop,1055232,4105.961089,13063
Smartphone,1135676,4436.234375,13155
Tablet,805445,3548.215859,13200


### <a id='toc2_3_'></a>[Multiple Aggregations](#toc0_)


You can apply different aggregations to different columns in a single operation:


In [33]:
multiple_aggs = df.groupby('Region').agg({
    'Revenue': ['sum', 'mean', 'median'],
    'Units': ['sum', 'mean'],
    'Sales': ['min', 'max']
})

multiple_aggs

Revenue                      Units           Sales      
            sum         mean  median   sum      mean   min   max
Region                                                          
East    1073138  4224.952756  3514.5  1277  5.027559   105  1495
North    925606  3989.681034  3234.0  1177  5.073276   100  1497
South   1096369  4403.088353  3563.0  1287  5.168675   104  1488
West    1020327  3850.290566  2884.0  1358  5.124528   102  1497

For more control over the resulting column names, you can use named aggregations:


In [34]:
named_aggs = df.groupby('Product').agg(
    Total_Revenue=('Revenue', 'sum'),
    Avg_Price=('Sales', 'mean'),
    Total_Units=('Units', 'sum'),
    Max_Single_Sale=('Revenue', 'max')
)

named_aggs

,Total_Revenue,Avg_Price,Total_Units,Max_Single_Sale
Product,,,,
Headphones,1119087,789.369231,1373,13473
Laptop,1055232,810.361868,1303,13239
Smartphone,1135676,814.746094,1355,13257
Tablet,805445,763.612335,1068,13302


You can also combine different types of aggregations:


In [35]:
combined_aggs = df.groupby(['Region', 'Product']).agg({
    'Revenue': ['sum', 'mean'],
    'Units': 'sum',
    'Sales': lambda x: x.max() - x.min()
}).round(2)

combined_aggs

Revenue          Units    Sales
                      sum     mean   sum <lambda>
Region Product                                   
East   Headphones  339165  4522.20   395     1390
       Laptop      258699  4042.17   344     1324
       Smartphone  300088  4763.30   322     1236
       Tablet      175186  3368.96   216     1312
North  Headphones  219987  3928.34   295     1365
       Laptop      242457  3974.70   298     1370
       Smartphone  275026  4825.02   313     1334
       Tablet      188136  3243.72   271     1396
South  Headphones  214237  3967.35   264     1332
       Laptop      318475  4485.56   358     1315
       Smartphone  336204  4872.52   385     1378
       Tablet      227453  4135.51   280     1372
West   Headphones  345698  4609.31   419     1386
       Laptop      235601  3862.31   303     1366
       Smartphone  224358  3348.63   335     1363
       Tablet      214670  3462.42   301     1384

These aggregation methods provide a powerful toolkit for summarizing your grouped data. By combining different aggregation functions, you can create comprehensive summaries that capture various aspects of your data, allowing for deeper insights and analysis.


Remember that the choice of aggregation method depends on the nature of your data and the specific insights you're trying to extract. Always consider the meaning and relevance of each aggregation in the context of your analysis.

## Grouping and Aggregating with Time Series Data

Time series data is common in many fields, including finance, sales, and scientific research. Pandas provides powerful tools for grouping and aggregating time-based data, allowing you to analyze trends over different time periods.

Let's explore various techniques for working with our sales time series data:

### Grouping by Time Periods

Pandas allows you to group by various time periods easily:

```python
# Group by month
monthly_sales = df.groupby(df['Date'].dt.to_period('M'))['Revenue'].sum()

monthly_sales.head()

# Group by quarter
quarterly_sales = df.groupby(df['Date'].dt.to_period('Q'))['Revenue'].sum()

quarterly_sales

# Group by year
yearly_sales = df.groupby(df['Date'].dt.year)['Revenue'].sum()

yearly_sales
```

### Resampling Time Series Data

Resampling is a convenient method for frequency conversion and time series analysis:

```python
# Set Date as index
df_indexed = df.set_index('Date')

# Resample to monthly data
monthly_revenue = df_indexed['Revenue'].resample('M').sum()

monthly_revenue.head()

# Resample to weekly data
weekly_revenue = df_indexed['Revenue'].resample('W').sum()

weekly_revenue.head()
```

### Rolling Window Calculations

Rolling (moving) window calculations are useful for analyzing trends:

```python
# 7-day rolling average of sales
rolling_7day = df_indexed['Revenue'].rolling(window=7).mean()

rolling_7day.head(10)

# 30-day rolling sum and standard deviation
rolling_30day = df_indexed['Revenue'].rolling(window=30).agg(['sum', 'std'])

rolling_30day.head()
```

### Grouping by Custom Time Periods

You can create custom time periods for grouping:

```python
# Group by day of week
day_of_week_sales = df.groupby(df['Date'].dt.day_name())['Revenue'].mean()

day_of_week_sales

# Group by week of year
week_of_year_sales = df.groupby(df['Date'].dt.isocalendar().week)['Revenue'].sum()

week_of_year_sales.head()
```

### Combining Time Grouping with Other Columns

You can combine time-based grouping with other categorical columns:

```python
# Monthly sales by product
monthly_product_sales = df.groupby([df['Date'].dt.to_period('M'), 'Product'])['Revenue'].sum().unstack()

monthly_product_sales.head()

# Quarterly sales by region
quarterly_region_sales = df.groupby([df['Date'].dt.to_period('Q'), 'Region'])['Revenue'].sum().unstack()

quarterly_region_sales
```

### 6. Year-over-Year Comparison

For year-over-year analysis:

```python
# Calculate year and month
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

# Year-over-year monthly comparison
yoy_comparison = df.groupby(['Month', 'Year'])['Revenue'].sum().unstack()

yoy_comparison
```

### 7. Cumulative Calculations

Cumulative calculations can show running totals over time:

```python
# Cumulative yearly revenue
cumulative_revenue = df.groupby(df['Date'].dt.to_period('Y'))['Revenue'].cumsum()

cumulative_revenue.head(10)
```

These techniques provide powerful ways to analyze time series data, allowing you to uncover trends, patterns, and seasonality in your sales data. By combining these time-based grouping and aggregation methods with other Pandas functions, you can perform sophisticated time series analysis to gain valuable insights into your data's temporal aspects.

Remember to choose the appropriate time granularity for your analysis based on the nature of your data and the insights you're seeking. Daily, weekly, monthly, or quarterly analyses can each provide different perspectives on your time series data.